# 특강: 성별에 따라 이름에서 나타나는 음성상징성

## 경로 연결하기


In [1]:
install.packages("googledrive")
library("googledrive")

if (file.exists("/usr/local/lib/python3.7/dist-packages/google/colab_ipython.py")){
  install.packages("R.utils")
  library("R.utils")
  library("httr")
  my_check <- function() {return(TRUE)}
  reassignInPackage("is_interactive", pkgName = "httr", my_check)
  options(rlang_interactive=TRUE)
}

drive_auth(use_oob = TRUE, cache = TRUE)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=380f246411ebcf098be79ef39fe4be45&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQVR4M0xZNG5fRjJVM0RSLVJQLU5UaUVkSmc1Qzl2RFhYTDhra1VHcjZMOGFWNm15MWdzNHROczRmaFBvaUdWT25HbkhoZyIsInN0YXRlIjoiMzgwZjI0NjQxMWViY2YwOThiZTc5ZWYzOWZlNGJlNDUifQ==


## 데이터 불러오기

In [2]:
dir()

[1] "outNameData.csv" "sample_data"

In [3]:
outNameData <- read.csv("outNameData.csv",head=T)
head(outNameData)

# 전체 데이터에서 수치형 자료만을 추출
outNameData <- outNameData[c(3,6,7,8,9,10,11,12,13,14,15)]
head(outNameData)
tail(outNameData)

,X,Gender,Label,Name_written,Name_spoken,segments,syllables,SCR,OCR,SCE,OCE,FVR,CVR,BVR,VE
,<int>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>
1,0,Male,0,adrian,eɪ d r i ə n,6,2,0.3333333,0.1666667,1,0,0.1666667,0.1666667,0.0000000,0
2,1,Male,0,eric,e r ɪ k,4,2,0.2500000,0.2500000,0,1,0.5000000,0.0000000,0.0000000,0
3,2,Male,0,dennis,d e n ɪ s,5,2,0.2000000,0.4000000,0,1,0.4000000,0.0000000,0.0000000,0
4,3,Male,0,marshall,m ɒ r ʃ ə l,6,2,0.5000000,0.1666667,1,0,0.0000000,0.1666667,0.1666667,0
5,4,Male,0,joshua,ʤ ɒ ʃ u ə,5,2,0.0000000,0.4000000,0,0,0.0000000,0.2000000,0.4000000,1
6,5,Male,0,michael,m aɪ k ə l,5,2,0.4000000,0.2000000,1,0,0.0000000,0.2000000,0.0000000,0


,Label,segments,syllables,SCR,OCR,SCE,OCE,FVR,CVR,BVR,VE
,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>
1,0,6,2,0.3333333,0.1666667,1,0,0.1666667,0.1666667,0.0000000,0
2,0,4,2,0.2500000,0.2500000,0,1,0.5000000,0.0000000,0.0000000,0
3,0,5,2,0.2000000,0.4000000,0,1,0.4000000,0.0000000,0.0000000,0
4,0,6,2,0.5000000,0.1666667,1,0,0.0000000,0.1666667,0.1666667,0
5,0,5,2,0.0000000,0.4000000,0,0,0.0000000,0.2000000,0.4000000,1
6,0,5,2,0.4000000,0.2000000,1,0,0.0000000,0.2000000,0.0000000,0


,Label,segments,syllables,SCR,OCR,SCE,OCE,FVR,CVR,BVR,VE
,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>
19995,1,3,1,0.3333333,0.0000000,0,0,0.3333333,0.3333333,0.00,1
19996,1,3,2,0.0000000,0.3333333,0,0,0.6666667,0.0000000,0.00,1
19997,1,4,2,0.2500000,0.2500000,0,0,0.2500000,0.0000000,0.25,1
19998,1,3,2,0.3333333,0.0000000,0,0,0.3333333,0.3333333,0.00,1
19999,1,4,2,0.2500000,0.5000000,0,1,0.0000000,0.0000000,0.00,0
20000,1,8,4,0.2500000,0.2500000,0,0,0.3750000,0.1250000,0.00,1


## 분석하기: 로지스틱 회귀분석

### 로지스틱 회귀분석 용어 정리
- Odds(p / (1-p)): 이벤트 발생할 확률을 이벤트가 발생하지 않을 확률로 나눈 값
- ex) p = 0.4, Odds = 0.4 / (1 - 0.4) = 0.667
<br><br>
- Logit(log(Odds)): Odds에 로그를 씌운 값
- ex) log(0.667) = -0.4049652

### 로지스틱 회귀분석 요약
- 분석의 대상이 되는 종속 변수가 성별처럼 class가 2개인 binominal 변수인 경우 사용한다.

![binominalCourve1](https://seongmin-mun.github.io/AjouUniversityCourse/2024/English%20Language%20and%20Literature/v2/image/binominalCourve1.png)

- 위의 그래프는 직선으로 관계를 나타낼 수 없는 비선형 데이터이기 때문에 아래의 그림처럼 로지스틱 함수 (시그모이드 함수; 초창기 딥러닝 모델에서 보다 정확한 학습 결과를 생성하기 위해 사용됨(i.e., 은닉층의 변화값))를 적용하여 분석을 해야함

![binominalCourve2](https://seongmin-mun.github.io/AjouUniversityCourse/2024/English%20Language%20and%20Literature/v2/image/binominalCourve2.png)

- 로지스틱 함수 공식: f(x) = 1 / (1 + exp(-x))
- 공식이 적용된 값은 0-1사이의 범위로 표현되며 y가 0.5보다 작은 경우 0으로, 큰 경우 1로 치환된다.

![binominalCourve3](https://seongmin-mun.github.io/AjouUniversityCourse/2024/English%20Language%20and%20Literature/v2/image/binominalCourve3.png)

### R을 사용한 로지스틱 회귀분석
- 로지스틱 회귀분석은 비선형의 데이터 분석이기 때문에 선형 회귀의 전제 조건인 정규성, 독립성, 등분산성 조건을 확인하지 않는다.
- 다만 전진 선택법(Forward) 혹은 후진 제거법(Backward) 방식의 단계별 선택법(Stepwise)를 적용한다.
- AIC(Akaike's Information Criterion)값이 최소가 되는 모델을 최적의 회귀모델로 생성한다.

In [4]:
# 종속변수: 성별; 독립변수: 분절음의 수, 음절의 수, (공명음, 장애음, 전설모음, 중설모음, 후설모음의 비율), (공명음, 장애음, 모음으로 끝나는지 여부)
logi_name <- glm(Label~.,family=binomial,data=outNameData)
summary(logi_name)


Call:
glm(formula = Label ~ ., family = binomial, data = outNameData)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -3.59063    0.27413 -13.098  < 2e-16 ***
segments    -0.53289    0.02197 -24.256  < 2e-16 ***
syllables    1.61530    0.04486  36.010  < 2e-16 ***
SCR          3.40547    0.34615   9.838  < 2e-16 ***
OCR          1.63137    0.33957   4.804 1.55e-06 ***
SCE          0.01092    0.20989   0.052   0.9585    
OCE         -0.47605    0.21255  -2.240   0.0251 *  
FVR          2.07285    0.20449  10.136  < 2e-16 ***
CVR          3.38179    0.26162  12.926  < 2e-16 ***
BVR         -2.31469    0.22801 -10.152  < 2e-16 ***
VE           1.37621    0.20808   6.614 3.75e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 27726  on 19999  degrees of freedom
Residual deviance: 21015  on 19989  degrees of freedom
AIC: 21037

Number of Fisher Scoring itera

In [5]:
bwd.logi_name <- step(logi_name,direction="backward")
summary(bwd.logi_name)

Start:  AIC=21036.62
Label ~ segments + syllables + SCR + OCR + SCE + OCE + FVR + 
    CVR + BVR + VE

            Df Deviance   AIC
- SCE        1    21015 21035
<none>            21015 21037
- OCE        1    21019 21039
- OCR        1    21038 21058
- VE         1    21066 21086
- SCR        1    21111 21131
- FVR        1    21117 21137
- BVR        1    21119 21139
- CVR        1    21183 21203
- segments   1    21622 21642
- syllables  1    22465 22485

Step:  AIC=21034.62
Label ~ segments + syllables + SCR + OCR + OCE + FVR + CVR + 
    BVR + VE

            Df Deviance   AIC
<none>            21015 21035
- OCR        1    21039 21057
- OCE        1    21088 21106
- SCR        1    21117 21135
- BVR        1    21121 21139
- FVR        1    21122 21140
- CVR        1    21192 21210
- segments   1    21623 21641
- VE         1    21947 21965
- syllables  1    22465 22483



Call:
glm(formula = Label ~ segments + syllables + SCR + OCR + OCE + 
    FVR + CVR + BVR + VE, family = binomial, data = outNameData)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -3.58284    0.22961 -15.604  < 2e-16 ***
segments    -0.53295    0.02195 -24.283  < 2e-16 ***
syllables    1.61538    0.04483  36.030  < 2e-16 ***
SCR          3.40938    0.33789  10.090  < 2e-16 ***
OCR          1.63480    0.33308   4.908  9.2e-07 ***
OCE         -0.48670    0.05724  -8.503  < 2e-16 ***
FVR          2.07497    0.20040  10.354  < 2e-16 ***
CVR          3.38469    0.25561  13.241  < 2e-16 ***
BVR         -2.31325    0.22631 -10.221  < 2e-16 ***
VE           1.36564    0.04549  30.023  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 27726  on 19999  degrees of freedom
Residual deviance: 21015  on 19990  degrees of freedom
AIC: 21035

Number of Fisher S

### 로지스틱 회귀분석 결과 해석
- Y = -3.58284 - 0.53295segments + 1.61538syllables + 3.40938SCR + 1.63480OCR - 0.48670OCE + 2.07497FVR + 3.38469CVR - 2.31325BVR + 1.36564VE
- 남성이 0이고 여성이 1이기 때문에 분석된 데이터에서 인물들의 이름은 남성이 확률이 높으며(상수항이 -3.58284임), 음절의 수가 많고, 공명음, 장애음, 전설모음, 중설모음의 비율이 높고 모음으로 끝나는 이름의 경우 여성일 확률이 높다.

## 모델 성능 평가
- Y = -3.58284 - 0.53295segments + 1.61538syllables + 3.40938SCR + 1.63480OCR - 0.48670OCE + 2.07497FVR + 3.38469CVR - 2.31325BVR + 1.36564VE

In [6]:
Log_odds <- predict(bwd.logi_name, newdata = outNameData)
Probability <- predict(bwd.logi_name, newdata = outNameData, type = 'response')
head(Probability)

1         2         3         4         5         6 
0.2260240 0.3380391 0.2078826 0.1987636 0.2234589 0.3431758

In [7]:
PREDICTED_C = ifelse(Probability > 0.5 , 1 , 0)
PREDICTED_C = as.factor(PREDICTED_C)
head(PREDICTED_C)

1 2 3 4 5 6 
0 0 0 0 0 0 
Levels: 0 1

### 성능 평가 값 도출: 정밀도, 재현율, F-score, 정확도


![ConfusionMatrix](https://seongmin-mun.github.io/AjouUniversityCourse/2024/English%20Language%20and%20Literature/v2/image/ConfusionMatrix.png)

In [8]:
ConfusionMatrix <- table(as.factor(outNameData$Label),as.factor(PREDICTED_C))
ConfusionMatrix

   
       0    1
  0 7800 2200
  1 2310 7690

- Precision(정밀도): 모델이 P로 판정한 데이터 중, 실제로 P인 데이터의 비율 - TP/(TP+FP)
- Recall(재현율): 실제 P인 데이터 중, 모델이 P로 판정한 데이터의 비율 - TP/(TP+FN)
- F1-score(F값): Precision*Recall의 조화평균(값은 0-1의 범위를 갖는다.) - 2*((Precision*Recall)/(Precision+Recall))
- Accuracy(정확도): 판별된 전체 데이터 중, TP와 TN으로 분류된 데이터의 비율 - (TP+TN)/(TP+FP+FN+TN)

In [9]:
TP <- ConfusionMatrix[1,1]; TP
FP <- ConfusionMatrix[1,2]; FP
FN <- ConfusionMatrix[2,1]; FN
TN <- ConfusionMatrix[2,2]; TN

[1] 7800

[1] 2200

[1] 2310

[1] 7690

In [10]:
Precision <- TP/(TP+FP); Precision
Recall <- TP/(TP+FN); Recall
F1 <- 2*((Precision*Recall)/(Precision+Recall)); F1
Accuracy <- (TP+TN)/(TP+FP+FN+TN); Accuracy

[1] 0.78

[1] 0.7715134

[1] 0.7757335

[1] 0.7745

### 모델 성능 평가 결과 해석
- 이름에서 나타나는 음운론적 특성을 사용하여 이름의 성별을 로지스틱 회귀 분석으로 분석하고, 생성된 모델을 실제 데이터에 적용한 결과, 77.5%의 정확도로 이름이 분류되는 것을 확인하였다.